# Table Joins to Look Up Large Lists of URLs in the Common Crawl


## A Short Introduction to URL indexes and SURT URLs

### The CDX Index

The Common Crawl CDX index [index.commoncrawl.org](https://index.commoncrawl.org/) allows to look up quickly whether a web page is contained in one of the monthly main crawls. Just a single request
```
curl -s 'https://index.commoncrawl.org/CC-MAIN-2021-10-index?url=https://commoncrawl.org/2021/02/january-2021-crawl-archive-now-available/&output=cdxj'
```
is sufficient to figure out that the announcement of the January crawl is included in the archives of the February crawl:
```
org,commoncrawl)/2021/02/january-2021-crawl-archive-now-available 20210303023607 {"url": "https://commoncrawl.org/2021/02/january-2021-crawl-archive-now-available/", "mime": "text/html", "mime-detected": "text/html", "status": "200", "digest": "6JKL2JMU4HFJMZAH65BZZET653JJP53I", "length": "6922", "offset": "250975924", "filename": "crawl-data/CC-MAIN-2021-10/segments/1614178365186.46/warc/CC-MAIN-20210303012222-20210303042222-00595.warc.gz", "languages": "eng", "encoding": "UTF-8"}
```

Every line in the index contains
* the URL search key aka. SURT ("Sort-friendly URI Reordering Transform")
* the capture time stamp
* a JSON object containing the URL, metadata and the location of a capture in the crawl archives

The so-called [zipnum sharded index](https://pywb.readthedocs.io/en/latest/manual/indexing.html?highlight=zipnum#zipnum-sharded-index) is sorted by the SURT URL key and split over 300 files. The index lines are gzip-compressed in blocks of 3000 lines. A secondary block index makes it possible to determine the block which should include a certain URL performing a quick binary search. Then the block is uncompressed to verify whether the URL is included, and if yes to extract the matched record(s).

Because the host name in the SURT URL is reversed (`subdomain.example.org` becomes `org,example,subdomain`), queries for all subdomains of a domain are possible by doing a prefix search. Other normalizations make the SURT URL even more powerful for prefix look-ups: eg. sorting of URL query parameters, removal of the protocol (`https://`) or stripping off the leading `www.` in host names. These variations in the URL often lead to redirects or duplicated content. However, they are not safe to be normalized in a hard way because in few cases web servers may serve different page content for different URLs sharing the same SURT representation.


### The Columnar Index

The [columnar index](https://commoncrawl.org/2018/03/index-to-warc-files-and-urls-in-columnar-format/) has the following differences compared to the CDX index:
- look-ups are not limited to the SURT URL, but are possible ony any provided column including content type (MIME), content languages and all URL parts (host name, registered domain, path, query)
- the columnar structure is efficient and saves costs if only a subset of the columns is accessed
- big data tools support SQL queries and aggregations ([Athena](https://aws.amazon.com/athena/)/[Presto](https://prestodb.io/), [Spark](https://spark.apache.org/), [Hive](https://hive.apache.org/))
- however, usage of a tool able to query the [Parquet file format](https://parquet.apache.org/) is required


### Index Partitioning

For practical reasons - because 3 billion pages contribute to an index size of about 300 GiB, every monthly crawl is indexed separately. So, you would typically use a CDX client (eg. [cdx-toolkit](https://pypi.org/project/cdx-toolkit/)) which takes the work to iterate over a list of indexes for a given time window.

Also the [columnar index](https://commoncrawl.org/2018/03/index-to-warc-files-and-urls-in-columnar-format/) stores every monthly crawl in a separate partition to alleviate the addition of a new crawl and also to target queries to individual crawls at zero computational cost.


## Efficiently Looking Up Large Lists of URLs

Back to our problem: how to look up a large list of URLs in the Common Crawl index...

Unless the URLs share a common prefix, for example a domain name, the CDX index isn't the best data structure to do this. The task translates into a series of look ups and does not scale well - the number of queries is equal to the number of URLs multiplied by the number of requested monthly indexes.

Similarly, runnning multiple queries over the columnar index isn't efficient. Instead we

1. create a support table which holds the list of URLs we want to look up
2. [Amazon Athena](https://aws.amazon.com/athena/) will do a table join to perform the intersection
3. the result table holds an export of the record coordinates but can be also utilized to further analyze or filter the result.

We use the SURT URLs to get a better recall without the need to artificially generate potential variants.


### Variants and Further Optimizations

- if possible try to further restrict the selection to perform the join on:
  - few monthly crawls
  - select on a crawl subset (successful fetches, robots.txt, 404s/redirects)
  - only URLs in certain top-level domains

  Simply add the restrictions to the WHERE-clause of the join query.

- if the URL list isn't really random but does belong to a set of host or domain names: performing the join on the columns `url_host_name` or `url_host_registered_domain` might be more efficient. Cf. [count-domains-alexa-top-1m.sql](https://github.com/commoncrawl/cc-index-table/blob/master/src/sql/examples/cc-index/count-domains-alexa-top-1m.sql) for a join on the domain name.

- if the list of URLs or domains is rather short, using the [Presto array functions](https://prestodb.io/docs/current/functions/array.html#array-functions) is much simpler than a table join:
```sql
SELECT *
FROM ccindex
WHERE contains(ARRAY ['example.com', '...'], url_host_registered_domain);
```

### Preparing the URL List

First, we need to prepare the URL list:
1. add the SURT URL as a column
2. convert it into a tabular format (Parquet)
3. upload the data to S3 and
4. import it as table into Athena


#### Adding the SURT URL

In [1]:
import surt

surt.surt('https://www.example.com/path/file?b=d&a=c')

'com,example)/path/file?a=c&b=d'

#### Writing Parquet

If the URL list isn't too big or is partitioned, the simplest way to convert into a Parquet table is to use [pandas](https://pandas.pydata.org/). We first add two columns to our example list holding the SURT URL and the host name:

In [2]:
from urllib.parse import urlparse

import pandas as pd
import surt

urls = ['http://example.org/',
        'https://www.example.org/path/file?b=d&a=c',
        'https://www.example.org/',
        'http://www.example.org:80',
        'http://commoncrawl.org/',
        'https://commoncrawl.org/2021/02/january-2021-crawl-archive-now-available/']

df = pd.DataFrame(data={'url': urls})

df['url_surtkey'] = df['url'].apply(surt.surt)
df['url_host_name'] = df['url'].apply(lambda url: urlparse(url).netloc.lower().lstrip('.'))

# reorder column and sort by SURT
df = df[['url_surtkey', 'url', 'url_host_name']].sort_values(['url_surtkey', 'url'])
df

,url_surtkey,url,url_host_name
4,"org,commoncrawl)/",http://commoncrawl.org/,commoncrawl.org
5,"org,commoncrawl)/2021/02/january-2021-crawl-ar...",https://commoncrawl.org/2021/02/january-2021-c...,commoncrawl.org
0,"org,example)/",http://example.org/,example.org
3,"org,example)/",http://www.example.org:80,www.example.org:80
2,"org,example)/",https://www.example.org/,www.example.org
1,"org,example)/path/file?a=c&b=d",https://www.example.org/path/file?b=d&a=c,www.example.org


and write the table as Parquet:

In [3]:
df.to_parquet('urls.parquet.gz', compression='gzip', index=False)

#### Import URL List as Athena Table

Now the URL table is uploaded to S3:
```
aws s3 cp urls.parquet.gz s3://mybucket/myjoin/urls/
```
Note: `mybucket` is a placeholder - create a bucket on your AWS account in the `us-east-1` region and change the bucket name accordingly.

Then we navigate to the [Athena query editor](https://console.aws.amazon.com/athena/home?region=us-east-1#/query-editor) and
- create a database "myjoin" by executing the following statement:
  ```sql
  CREATE DATABASE myjoin;
  ```
- register the table "urls":
  ```sql
  CREATE EXTERNAL TABLE IF NOT EXISTS myjoin.urls (
    `url_surtkey`   string,
    `url`           string,
    `url_host_name` string
  )
  ROW FORMAT SERDE 'org.apache.hadoop.hive.ql.io.parquet.serde.ParquetHiveSerDe'
  WITH SERDEPROPERTIES (
    'serialization.format' = '1'
  ) LOCATION 's3://mybucket/myjoin/urls/'
  TBLPROPERTIES ('has_encrypted_data'='false');
  ```
- and verify whether the table is imported properly and contains the expected number of rows
  ```sql
  SELECT * FROM myjoin.urls limit 10;
  
  SELECT COUNT(*) FROM myjoin.urls;
  ```

### Importing the Common Crawl Index Table

See [instructions](https://commoncrawl.org/2018/03/index-to-warc-files-and-urls-in-columnar-format/) and the [up-to-date CREATE TABLE statement](https://github.com/commoncrawl/cc-index-table/blob/master/src/sql/athena/cc-index-create-table-flat.sql). If done the index table should be ready under the name "ccindex.ccindex".


### Joining the Tables

Now we are ready to join our URL table with Common Crawl's main table and create a new table with all captures for our URLs:

```sql
CREATE TABLE myjoin.captures_cc
WITH (external_location = 's3://mybucket/myjoin/captures_cc/',
      format = 'PARQUET',
      parquet_compression = 'GZIP')
AS SELECT cc.url_surtkey     AS url_surtkey,
       cc.url                AS url,
       my.url                AS my_url,
       cc.url_host_name      AS url_host_name,
       cc.warc_filename      AS warc_filename,
       cc.warc_record_offset AS warc_record_offset,
       cc.warc_record_length AS warc_record_length,
       cc.crawl              AS crawl,
       cc.subset             AS subset
FROM ccindex.ccindex AS cc
  INNER JOIN myjoin.urls AS my
  ON my.url_surtkey = cc.url_surtkey
WHERE cc.crawl = 'CC-MAIN-2021-10'
  AND cc.subset = 'warc'
  AND cc.url_host_tld = 'org'
```

Because all our sample URLs are in the `.org` TLD we can optimize the join which makes it succeed fast - the join query succeeded within 7 seconds scanning only 400 MiB of data.

We defined Parquet as output format of the joined table. That's ideal to further inspect the joined data. But Athena supports also other [output formats](https://docs.aws.amazon.com/athena/latest/ug/create-table-as.html#ctas-table-properties) (JSON, AVRO, ORC and other).

Let's look what we have in the joined table:
```sql 
SELECT url_surtkey, url, my_url FROM myjoin.captures_cc LIMIT 100;
```

Not all URLs are found, some URLs have multiple captures and the captured URLs may differ from those in the list:

|url_surtkey             |url                     |my_url                       |
|:-----------------------|:-----------------------|:----------------------------|
|org,commoncrawl)/2021/02/january-2021-crawl-archive-now-available|https://commoncrawl.org/2021/02/january-2021-crawl-archive-now-available/|https://commoncrawl.org/2021/02/january-2021-crawl-archive-now-available/|
|org,commoncrawl)/       |http://commoncrawl.org/ |http://commoncrawl.org/      |
|org,example)/           |https://example.org/    |https://www.example.org/     |
|org,example)/           |https://example.org/    |http://example.org/          |
|org,example)/           |https://example.org/    |http://www.example.org:80    |